In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.applications as apps
import pandas as pd
import kagglehub
from pathlib import Path
import numpy as np
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import train_test_split
import gc


2025-04-08 23:11:38.188807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744150298.205448  152577 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744150298.209734  152577 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744150298.223442  152577 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744150298.223518  152577 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744150298.223522  152577 computation_placer.cc:177] computation placer alr

In [2]:
def set_memory_growth():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
      try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
      except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

def set_memory_limit(memory_limit):
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit)]
        )

    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

set_memory_limit(4096)
#set_memory_growth()
keras.mixed_precision.set_global_policy("mixed_float16")

pd.set_option('display.max_columns', None)

1 Physical GPU, 1 Logical GPUs


I0000 00:00:1744150302.848582  152577 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4096 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [3]:
SEED = 42
BATCH_SIZE = 8
PATIENCE = 5

saved_models_dir = Path("../saved_models")
saved_models_dir.mkdir(parents=True, exist_ok=True)
histories_dir = Path("../histories")
histories_dir.mkdir(parents=True, exist_ok=True)

# Create Datasets

In [4]:
# Download latest version of data
## Use the next 2 lines if not downloaded before
# image_dir = kagglehub.dataset_download("bloox2/fieldplant")
# image_dir = Path(image_dir) / "train"

# My data has been cached
image_dir = "/home/ruairi/.cache/kagglehub/datasets/bloox2/fieldplant/versions/1/train"
print("Path to dataset files:", image_dir)

Path to dataset files: /home/ruairi/.cache/kagglehub/datasets/bloox2/fieldplant/versions/1/train


# Create Models

In [5]:
def get_model_and_preprocessing(app_name):
    app = getattr(keras.applications, app_name)
    model_name = dir(app)[0]
    model = getattr(app, model_name)
    input_shape = model().input_shape[1:]
    model = model(include_top=False, input_shape=input_shape)
    model.trainable = False
    preprocessing = getattr(app, "preprocess_input")
    return model, preprocessing

In [6]:
def get_pre_classifier_layers(model_name):
    if "vgg" in model_name:
        return keras.Sequential([
            keras.layers.GlobalAveragePooling2D(),
            keras.layers.Dense(1024, activation="relu"),
            keras.layers.Dense(1024, activation="relu") ])
    else:
        return keras.Sequential([keras.layers.GlobalAveragePooling2D()])

    

In [ ]:
# if(cnn_model=='mobilenet'):

#         ### START CODE HERE

#         base_model = MobileNetV2(input_shape=input_shape,
#                                                        include_top=False, # <== Important!!!!
#                                                     weights=pretrained_weights) # From imageNet

#         # freeze the base model by making it non trainable
#         base_model.trainable = False

#        # create the input layer (Same as the imageNetv2 input size)
#         inputs = tf.keras.Input(shape=input_shape)
#         print(inputs)

#         # apply data augmentation to the inputs
#         x = (inputs)

#         # data preprocessing using the same weights the model was trained on
#         x = tf.keras.applications.mobilenet_v2.preprocess_input(x)

#         # set training to False to avoid keeping track of statistics in the batch norm layer
#         x = base_model(x, training=False)

#         # add the new Binary classification layers
#         #x=base_model.output
#         # use global avg pooling to summarize the info in each channel
#         x = tf.keras.layers.GlobalAveragePooling2D()(x)

#         # include dropout with probability of 0.2 to avoid overfitting
#         #x = None(None)(x)
#         x = tf.keras.layers.Dropout(dropout_value, noise_shape=None, seed=None)(x)

#         # use a prediction layer with one neuron (as a binary classifier only needs one)
#         x = tf.keras.layers.Flatten()(x)

#         # use a prediction layer with one neuron (as a binary classifier only needs one)
#         outputs = tf.keras.layers.Dense(number_classes, activation='softmax')(x)
#         ### END CODE HERE

#         model = tf.keras.Model(inputs, outputs)

#     elif(cnn_model=='vgg'):
#         """
#         Compiles a model integrated with VGG16 pretrained layers

#         input_shape: tuple - the shape of input images (width, height, channels)
#         n_classes: int - number of classes for the output layer
#         optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
#         fine_tune: int - The number of pre-trained layers to unfreeze.
#                     If set to 0, all pretrained layers will freeze during training
#         """

#         # Pretrained convolutional layers are loaded using the Imagenet weights.
#         # Include_top is set to False, in order to exclude the model's fully-connected layers.
#         conv_base = VGG16(include_top=False,
#                          weights=pretrained_weights,
#                          input_shape=input_shape)

#         conv_base.trainable=False #The base model must not be trainable

#         # Create a new 'top' of the model (i.e. fully-connected layers).
#         # This is 'bootstrapping' a new top_model onto the pretrained layers.
#         top_model = conv_base.output
#         top_model = Flatten(name="flatten")(top_model)
#         top_model = Dense(4096, activation='relu')(top_model)
#         top_model = Dense(1072, activation='relu')(top_model)
#         top_model = Dropout(dropout_value)(top_model)
#         output_layer = Dense(number_classes, activation='softmax')(top_model)

#         # Group the convolutional base and new fully-connected layers into a Model object.
#         model = Model(inputs=conv_base.input, outputs=output_layer)

#     elif(cnn_model=="inceptionv3"):

#         base_model = tensorflow.keras.applications.inception_v3.InceptionV3(
#             input_shape=input_shape,
#             include_top=False,
#             weights=pretrained_weights)

#         last_layer = base_model.get_layer('mixed7')
#         print('last layer output shape:', last_layer.output_shape)
#         last_output = last_layer.output

#         # Flatten the output layer to 1 dimension
#         x = tensorflow.keras.layers.Flatten()(last_output)
#         # Add a fully connected layer with 1,024 hidden units and ReLU activation
#         x = tensorflow.keras.layers.Dense(1024, activation='relu')(x)
#         # Add a dropout rate of 0.2
#         x = tensorflow.keras.layers.Dropout(dropout_value)(x)
#         # Add a final sigmoid layer for classification
#         x = tensorflow.keras.layers.Dense(number_classes, activation='softmax')(x)

#         # Configure and compile the model
#         model = tensorflow.keras.Model(base_model.input, x)
#     elif(cnn_model=="inceptionresnet"):

#         base_model=InceptionResNetV2(
#             weights='imagenet',
#             include_top=False,
#             input_shape=input_shape)

#         base_model.trainable=False #The base model must not be trainable

#         x = base_model.output
#         x = keras.layers.GlobalAveragePooling2D()(x)
#         x = keras.layers.Dense(1024, activation = 'relu')(x)
#         x = keras.layers.Dropout(dropout_value)(x)
#         predictions = keras.layers.Dense(number_classes, activation='softmax')(x)
#         model = Model(inputs=base_model.input, outputs=predictions)

#     return model

In [7]:
def build_model(app_name, activation, num_classes):
    model, preprocessing = get_model_and_preprocessing(app_name)
    pre_classifier_layers = get_pre_classifier_layers(model.name)

    inputs = keras.Input(shape=model.input_shape[1:])
    x = preprocessing(inputs)
    x = model(x, training=False)
    x = pre_classifier_layers(x)
    outputs = keras.layers.Dense(num_classes, activation=activation, name="classifier_layer")(x)

    model_name = model.name
    model = keras.Model(inputs, outputs, name=model_name)

    return model

In [8]:
def get_hyperparameters(methodology):
    multilabel = methodology == "multilabel"
    
    methodologies = ["multiclass", "multilabel"]
    losses = ["categorical_crossentropy", "binary_crossentropy"]
    activation = ["softmax", "sigmoid"]
    metrics = ["categorical_accuracy", "binary_accuracy"]

    idx = methodologies.index(methodology)

    metrics = [metrics[idx]]
    f1_score_weighted = keras.metrics.F1Score(average="weighted", threshold=0.5 if multilabel else None, name="f1_score_weighted", dtype=None)
    f1_score_per_class = keras.metrics.F1Score(average=None, threshold=0.5 if multilabel else None, name="f1_score_per_class", dtype=None)
    metrics.append(f1_score_weighted)
    metrics.append(f1_score_per_class)
        
    hyperparams = [losses[idx], activation[idx], metrics]
    
    return hyperparams

In [9]:
def print_model_info(model):
    print()
    print("print_model_info() start")
    compile_config = model._compile_config.config
    optimizer = compile_config['optimizer'].get_config()
    classifier_activation = model.get_layer(name="classifier_layer").activation.__name__

    print("Model name:", model.name)
    print("Input shape:", model.input_shape)
    print("Optimizer name:", optimizer['name'], "learning_rate:", np.round(optimizer['learning_rate'], 6))
    print("Loss:", compile_config['loss'])
    print("Metrics:")
    for metric in compile_config['metrics']:
        print(metric if isinstance(metric, str) else metric.get_config())
    print("Classifier layer activation function:", classifier_activation)
    print()
    print(model.summary(expand_nested=True, show_trainable=True))
    print()

    print("print_model_info() end")
    print()


In [10]:
def get_callbacks(best_epoch=None, model_name=None, fine_tuning=False):
    if fine_tuning:
        baseline_loss = best_epoch.val_loss.values[0]
        print("Previous best epoch (starting counting from 0):", best_epoch.epoch.values[0])
        print("Previous best epoch (starting counting from 1, as per training loop):", best_epoch.epoch.values[0] +1)

        print("Baseline val_loss:", baseline_loss)
    cbs = [
        keras.callbacks.EarlyStopping(
            patience=PATIENCE, restore_best_weights=True,
            baseline=None if not fine_tuning else baseline_loss,
            verbose=1),
        keras.callbacks.ModelCheckpoint(
            filepath=f"{saved_models_dir}/{model_name}.keras",
            save_best_only=True, monitor="val_loss",
            verbose=1,
            initial_value_threshold=None if not fine_tuning else baseline_loss
        )]
    return cbs


# GET DATASETS

In [11]:
def get_dataframe(filtered=False, sample=False):
    filename = "filtered" if filtered else "unfiltered"
    all_csv_files = list(Path("../data").glob("*"))
    csv_file = [csv for csv in all_csv_files if filename in csv.name][0]
    df = pd.read_csv(csv_file)
    print()

    if sample:
        print("Using sampled DF")
        df = df.sample(frac=0.5)
        df = df.loc[(df!=0).any(axis=1)]
    print("Df shape:", df.shape)
    print()
    return df   

In [12]:
def get_train_test_splits(df, filtered=False, test_size=0.2):
    col_names = list(df.columns)
    split_fn = get_stratified_splits if filtered else get_nonstratified_splits
    (X_train, X_test, X_val, y_train, y_test, y_val) = split_fn(df, test_size=test_size)
    train_df = pd.merge(X_train, y_train, left_index=True, right_index=True)
    test_df = pd.merge(X_test, y_test, left_index=True, right_index=True)
    val_df = pd.merge(X_val, y_val, left_index=True, right_index=True)
    
    train_df.columns = col_names
    test_df.columns = col_names
    val_df.columns = col_names

    return train_df, test_df, val_df
    
def get_stratified_splits(df, test_size=0.2):
    columns = list(df.columns)
    X = df.filename.to_frame().to_numpy()
    y = df.drop(columns=["filename"]).to_numpy()

    X_train, y_train, X_test_val, y_test_val = iterative_train_test_split(X, y, test_size=0.2)
    X_test, y_test, X_val, y_val = iterative_train_test_split(X_test_val, y_test_val, test_size=0.5)
    datasets = (X_train, X_test, X_val, y_train, y_test, y_val)
    datasets = [pd.DataFrame(dataset) for dataset in datasets]
    return tuple(datasets)

def get_nonstratified_splits(df, test_size=0.2):
    X = df.filename
    y = df.drop(columns=["filename"])
    X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=SEED)
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=SEED)

    return (X_train, X_test, X_val, y_train, y_test, y_val)

In [13]:
def decode_img(filename, img_size):
    filepath = str(image_dir) + "/" + filename
    img = tf.io.read_file(filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_size)
    return img
    
def process_dataset(filename, labels, img_size):
    img = decode_img(filename, img_size=img_size)
    return img, labels

def configure_datasets_for_performance(datasets, shuffle=False, batch_size=BATCH_SIZE):
    configured_datasets = []
    ds_sizes = [int(ds.cardinality().numpy()) for ds in datasets]
    print("Ds sizes:", ds_sizes)
    print("NP argmax:", np.argmax(ds_sizes))
    for i, dataset in enumerate(datasets):
        if int(dataset.cardinality().numpy()) == ds_sizes[np.argmax(ds_sizes)]:
            print(f"Shuffling dataset {i}")
            print()
            dataset = dataset.shuffle(buffer_size=dataset.cardinality(), reshuffle_each_iteration=True)
        dataset = dataset.batch(batch_size=batch_size, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.cache()
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
        configured_datasets.append(dataset)
    return tuple(configured_datasets)

In [14]:
def datasets_from_dataframes(img_size, splits=None):
    datasets = []
    ds_names = ["train", "val", "test"]
    for i, split in enumerate(splits):
        img = split.filename
        labels = split.drop(columns=["filename"])
        
        dataset = tf.data.Dataset.from_tensor_slices((img, labels))
        print(ds_names[i], "length:", len(dataset))

        dataset = dataset.map(lambda x,y: process_dataset(x,y, img_size))
        print(ds_names[i], dataset.element_spec)
        print()
        
        datasets.append(dataset)
    return tuple(datasets)

In [15]:
def get_datasets(img_size, df, test_size=0.2):
    splits = get_train_test_splits(df, test_size=test_size)
    datasets = datasets_from_dataframes(img_size, splits=splits)
    return datasets

# CREATE MODELS

In [16]:
def freeze_model(model):
    print()
    print("freeze_model() start")

    base_layer_name = ""
    num = 0
    if "mobilenet" in model.name:
        base_layer_name = "mobilenetv2_1.00_224"
        num = 120
    elif "vgg16" in model.name:
        base_layer_name = "vgg16"
        num = 14
    elif "inception_v3" in model.name:
        base_layer_name = "inception_v3"
        num = 172
    elif "resnet" in model.name:
        base_layer_name = "inception_resnet_v2"
        num = 516

    print(f"Freeze layers up to layer {num}")
    for layer in model.get_layer(name=base_layer_name).layers[:num]:
        layer.trainable = False
    for layer in model.get_layer(name=base_layer_name).layers[num:]:
        layer.trainable = True

    print("freeze_model() end")
    print()

In [17]:
def train_model(best_epoch=None, model=None, train_ds=None, val_ds=None, epochs=100, loss=None, metrics=[], fine_tuning=False):
    print()
    print("train_model() start")

    string_suffix = "FT" if fine_tuning else "CLF"
    print("Training -", string_suffix)

    if fine_tuning:
        freeze_model(model)
        
    compile_model(model, loss=loss, metrics=metrics, fine_tuning=fine_tuning)
    print_model_info(model)
       
    cbs = get_callbacks(model_name=model.name, fine_tuning=fine_tuning, best_epoch=best_epoch)

    # +1 to best epoch so that it matches the number given in the training loop.  (So best=7 becomes best=8)
    # +1 again so that it begins training from the epoch following the previous best. (So inital_epoch=8 becomes inital_epoch=9)
    initial_epoch = 0 if not fine_tuning else best_epoch.epoch.values[0] + 1 + 1

    print(f"Starting training at epoch {initial_epoch}. (starting counting from 0, as per df)" )
    print(f"Starting training at epoch {initial_epoch + 1}. (starting counting from 1, as per training loop)" )
    print()

    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=cbs, initial_epoch=initial_epoch)
    history_df = pd.DataFrame(history.history)
    history_df['model'] = model.name
    history_df['epoch'] = history.epoch
    history_df['type'] = string_suffix
    
    save_filename = f"{str(histories_dir)}/{model.name}_{string_suffix}.csv"
    print()
    print("Saving file as:", save_filename)
    history_df.to_csv(save_filename, index=False)  
    
    best_epoch = history_df.loc[history_df.val_loss == history_df.val_loss.min()]
    print(f"Best epoch number (starting from 0, as per df): {best_epoch.epoch.values[0]}")
    print(f"Best epoch number (starting from 1, as per training loop): {best_epoch.epoch.values[0] + 1}")
    print()


    display(history_df)

    print()
    print("train_model() end")

    return best_epoch
    

In [18]:
def compile_model(model, loss=None, metrics=[], fine_tuning=False):
    print()
    print("compile_model() start")
    opt = keras.optimizers.Adam
    lr = (float(opt().learning_rate) / 10) if fine_tuning else float(opt().learning_rate)
    
    model.compile(
        loss=loss,
        optimizer=keras.optimizers.Adam(learning_rate=lr, gradient_accumulation_steps=4),
        metrics=metrics
    )
    print("compile_model() end")
    print()

In [20]:
def run_all(epochs=5, sample=True):
    print()
    print("run_all() start")

    i = 0
    num_models = 0

    ## Use for training the entire loop
    # app_names = ["mobilenet_v2", "vgg16", "inception_v3", "inception_resnet_v2"]
    # methodologies = ["multiclass", "multilabel"]
    # filter_options = [False, True]

    # # Use for smaller combinations
    app_names = ["mobilenet_v2"]
    methodologies = ["multiclass", "multilabel"]
    filter_options = [False, True]


    # Get total count of models to be training
    for filtered in filter_options:
        for methodology in methodologies:
            if not filtered and (methodology == "multilabel"):
                continue
            for app_name in app_names:
                num_models += 1

    # Training Loop
    for filtered in filter_options:
        for methodology in methodologies:
            if not filtered and (methodology == "multilabel"):
                continue
                
            loss, activation, metrics = get_hyperparameters(methodology)
    
            df = get_dataframe(filtered=filtered, sample=sample)
            num_classes = len(df.columns[1:])
            
            for app_name in app_names:
                i += 1
                model = build_model(app_name, activation=activation, num_classes=num_classes)
                model.name = model.name + "_" + ("filtered" if filtered else "unfiltered") + "_" + methodology
                img_size = model.input_shape[1:3]
                datasets = get_datasets(img_size, df=df)
                train_ds, test_ds, val_ds = configure_datasets_for_performance(datasets)

                  # Train top Classifier
                for fine_tuning in [False, True]:
                    print()
                    print(f"Model: {i} of {num_models}")
                    print("app_name:", app_name)
                    print("filtered:", filtered)
                    print("methodology:", methodology)
                    
                    best_epoch = train_model(model=model, best_epoch=None if not fine_tuning else best_epoch,
                                             train_ds=train_ds, val_ds=val_ds,
                                             epochs=epochs, loss=loss, metrics=metrics,
                                             fine_tuning=fine_tuning)

                                   
                # Clear memory
                del model
                gc.collect()
                tf.keras.backend.clear_session()

                print(); print(); print(); print();
                print("BEGINNING NEXT MODEL IF EXISTS...")

                # # Train top Classifier
                # best_epoch = train_model(model=model, train_ds=train_ds, val_ds=val_ds, epochs=epochs, loss=loss, metrics=metrics, fine_tuning=False)
                # # Fine Tuning
                # train_model(best_epoch=best_epoch, model=model, train_ds=train_ds, val_ds=val_ds, epochs=epochs, loss=loss, metrics=metrics, fine_tuning=True)

                
    print("run_all() end")
    print()

In [21]:
model = run_all(epochs=100, sample=False)


run_all() start

Df shape: (5156, 28)

train length: 4124
train (TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(27,), dtype=tf.int64, name=None))

val length: 516
val (TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(27,), dtype=tf.int64, name=None))

test length: 516
test (TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(27,), dtype=tf.int64, name=None))

Ds sizes: [4124, 516, 516]
NP argmax: 0
Shuffling dataset 0


Model: 1 of 3
app_name: mobilenet_v2
filtered: False
methodology: multiclass

train_model() start
Training - CLF

compile_model() start
compile_model() end


print_model_info() start
Model name: mobilenetv2_1.00_224_unfiltered_multiclass
Input shape: (None, 224, 224, 3)
Optimizer name: adam learning_rate: 0.001
Loss: categorical_crossentropy
Metrics:
categorical_accuracy
{'name': 'f1_score_weighted', 'dtype': 'float32', 'average': 'weighted', 'threshold': None}
{'name': 'f1_scor

Model: "mobilenetv2_1.00_224_unfiltered_multiclass"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_2 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ true_divide (TrueDivide)    │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ subtract (Subtract)         │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ mobilenetv2_1.00_224        │ (None, 7, 7, 1280)    │  2,257,984 │   N   │
│ (Functional)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ input_layer_1          │ (None, 224, 224, 3)   │          0 │   -   │
│ (InputLayer)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ Conv1 (Conv2D)         │ (None, 112, 112, 32)  │        864 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ bn_Conv1               │ (None, 112, 112, 32)  │        128 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ Conv1_relu (ReLU)      │ (None, 112, 112, 32)  │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └                        │ (None, 112, 112, 32)  │        288 │   N   │
│ expanded_conv_depthwise     │                       │            │       │
│ (DepthwiseConv2D)           │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └                        │ (None, 112, 112, 32)  │        128 │   N   │
│ expanded_conv_depthwise_BN  │                       │            │       │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └                        │ (None, 112, 112, 32)  │          0 │   -   │
│ expanded_conv_depthwise_re… │                       │            │       │
│ (ReLU)                      │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ expanded_conv_project  │ (None, 112, 112, 16)  │        512 │   N   │
│ (Conv2D)                    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └                        │ (None, 112, 112, 16)  │         64 │   N   │
│ expanded_conv_project_BN    │                       │            │       │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ block_1_expand         │ (None, 112, 112, 96)  │      1,536 │   N   │
│ (Conv2D)                    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ block_1_expand_BN      │ (None, 112, 112, 96)  │        384 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ block_1_expand_relu    │ (None, 112, 112, 96)  │          0 │   -   │
│ (ReLU)                      │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ block_1_pad            │ (None, 113, 113, 96)  │          0 │   -   │
│ (ZeroPadding2D)             │                       │            │     

 Total params: 2,292,571 (8.75 MB)

 Trainable params: 34,587 (135.11 KB)

...

 Non-trainable params: 2,257,984 (8.61 MB)

None

print_model_info() end

Starting training at epoch 0. (starting counting from 0, as per df)
Starting training at epoch 1. (starting counting from 1, as per training loop)

Epoch 1/100


2025-04-08 23:21:03.173116: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:42: Filling up shuffle buffer (this may take a while): 2822 of 4123
2025-04-08 23:21:08.241521: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.


516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - binary_accuracy: 0.9345 - f1_score_per_class: 0.0833 - f1_score_weighted: 0.3141 - loss: 0.1772
Epoch 1: val_loss improved from inf to 0.07451, saving model to ../saved_models/mobilenetv2_1.00_224_filtered_multilabel.keras
516/516 ━━━━━━━━━━━━━━━━━━━━ 39s 35ms/step - binary_accuracy: 0.9345 - f1_score_per_class: 0.0834 - f1_score_weighted: 0.3143 - loss: 0.1771 - val_binary_accuracy: 0.9740 - val_f1_score_per_class: 0.1728 - val_f1_score_weighted: 0.5443 - val_loss: 0.0745
Epoch 2/100
513/516 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - binary_accuracy: 0.9772 - f1_score_per_class: 0.2166 - f1_score_weighted: 0.6054 - loss: 0.0664
Epoch 2: val_loss improved from 0.07451 to 0.06221, saving model to ../saved_models/mobilenetv2_1.00_224_filtered_multilabel.keras
516/516 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - binary_accuracy: 0.9772 - f1_score_per_class: 0.2169 - f1_score_weighted: 0.6056 - loss: 0.0664 - val_binary_accuracy: 0.9771 - val_f1_score_per_class:

KeyboardInterrupt: 